In [68]:
import pandas as pd
import sqlite3

In [69]:
con = sqlite3.connect("../data/checking-logs.sqlite")
cur = con.cursor()
schema = pd.read_sql("PRAGMA table_info(test);", con)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [70]:
test_10 = pd.read_sql("SELECT * FROM test LIMIT 10", con)
test_10

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [71]:
df_min = pd.read_sql("""
            SELECT
                uid,
                MIN((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as min_diff
            FROM test t JOIN deadlines d ON d.labs = t.labname
            WHERE t.labname != 'project1'
            """
            ,con)
df_min

,uid,min_diff
0,user_30,-202


In [72]:
df_max = pd.read_sql("""
            SELECT
                uid,
                MAX((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as max_diff
            FROM test t JOIN deadlines d ON d.labs = t.labname
            WHERE t.labname != 'project1'
            """
            ,con)
df_max

,uid,max_diff
0,user_25,-2


In [73]:
df_avg = pd.read_sql("""
            SELECT
                AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as avg_diff
            FROM test t JOIN deadlines d ON d.labs = t.labname
            WHERE t.labname != 'project1'
            """
            ,con)
df_avg

,avg_diff
0,-89.125


In [74]:
views_diff = pd.read_sql(""" 
            SELECT 
                t.uid,
                AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as avg_diff,
                COUNT (*) AS pageviews
            FROM test t LEFT JOIN deadlines d ON d.labs = t.labname
            WHERE t.labname != 'project1'
            GROUP BY uid
            """,
            con)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,5
1,user_10,-74.800000,5
2,user_14,-159.000000,3
3,user_17,-61.600000,5
4,user_18,-5.666667,3
5,user_19,-98.750000,4
6,user_21,-95.500000,4
7,user_25,-92.600000,5
8,user_28,-86.400000,5
9,user_3,-105.400000,5


In [77]:
con.close()

In [78]:
views_diff.isnull().sum()

uid          0
avg_diff     0
pageviews    0
dtype: int64

In [80]:
print(views_diff.dtypes)

uid           object
avg_diff     float64
pageviews      int64
dtype: object


In [81]:
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,5
1,user_10,-74.800000,5
2,user_14,-159.000000,3
3,user_17,-61.600000,5
4,user_18,-5.666667,3
5,user_19,-98.750000,4
6,user_21,-95.500000,4
7,user_25,-92.600000,5
8,user_28,-86.400000,5
9,user_3,-105.400000,5


In [82]:
corr_matrix = views_diff[['avg_diff', 'pageviews']].corr()
print("Матрица корреляций:")
print(corr_matrix)

Матрица корреляций:
           avg_diff  pageviews
avg_diff   1.000000   0.117685
pageviews  0.117685   1.000000


In [84]:
# Можно сделать вывод, что корреляции между средней дельтой и количеством просмотров страницы нет
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,0.117685
pageviews,0.117685,1.000000
